####Explode the columns as value as string

In [0]:
#Importing all the required lib
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Create DataFrame") \
    .getOrCreate()

# Define schema for the DataFrame
schema = StructType([
    StructField("ID", IntegerType(), True),
    StructField("Name", StringType(), True)
])

# Data
data = [
    (11, "Arabinda,Shyam"),
    (12, "Sunil,Ram,Hari"),
    (13, "Jonh"),
    (14, "Derell,Raghu"),
    (15, "Jonh")
]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show DataFrame
df.show()


+---+--------------+
| ID|          Name|
+---+--------------+
| 11|Arabinda,Shyam|
| 12|Sunil,Ram,Hari|
| 13|          Jonh|
| 14|  Derell,Raghu|
| 15|          Jonh|
+---+--------------+



In [0]:
df=df.withColumn("Name",explode_outer(split(col("Name"),",")))
df.show()

+----+--------+
|  ID|    Name|
+----+--------+
|null|       B|
|null|    null|
|null|    null|
|  11|Arabinda|
|  12|   Shyam|
|  13|   Sunil|
|  14|     Ram|
|  15|    Hari|
|  16|    John|
+----+--------+



####Skip the First n rows in pysaprk

In [0]:



# Define schema for the DataFrame
schema = StructType([
    StructField("ID", IntegerType(), True),
    StructField("Name", StringType(), True)
])


# Data
data = [
    (None, "B"),
    (None, None),
    (None, None),
    (11, "Arabinda"),
    (12, "Shyam"),
    (13, "Sunil"),
    (14, "Ram"),
    (15, "Hari"),
    (16, "John")
]


# Create DataFrame for data1
df = spark.createDataFrame(data, schema)



In [0]:
df.show()

+----+--------+
|  ID|    Name|
+----+--------+
|null|       B|
|null|    null|
|null|    null|
|  11|Arabinda|
|  12|   Shyam|
|  13|   Sunil|
|  14|     Ram|
|  15|    Hari|
|  16|    John|
+----+--------+



####Approach-1

In [0]:
 #df=df.rdd.zipWithIndex().filter(lambda x: x[1] >= 3).map(lambda x:x[0]).toDF())

df_1=df.rdd.zipWithIndex() #Index generating
df_2=df_1.filter(lambda x: x[1] >= 3) # filter the index to be >=3
df_f=df_2.map(lambda x:x[0]).toDF() # map transformation one value to be output 
 #map(lambda x: x[0]).toDF().show()
 df_f.show()

+---+--------+
| ID|    Name|
+---+--------+
| 11|Arabinda|
| 12|   Shyam|
| 13|   Sunil|
| 14|     Ram|
| 15|    Hari|
| 16|    John|
+---+--------+



####Approach-2

In [0]:
# Add a row number column
df_00 = df.withColumn("row_id", monotonically_increasing_id())

windowSpec = Window.orderBy("row_id")
df_11 = df_00.withColumn("row_num", row_number().over(windowSpec))
df_final=df_11[df_11["row_num"]>=4].drop("row_id","row_num")

df_final.show()

+---+--------+
| ID|    Name|
+---+--------+
| 11|Arabinda|
| 12|   Shyam|
| 13|   Sunil|
| 14|     Ram|
| 15|    Hari|
| 16|    John|
+---+--------+



In [0]:
df.show(#)

+----+--------+
|  ID|    Name|
+----+--------+
|null|    null|
|null|    null|
|  11|Arabinda|
|  12|   Shyam|
|  13|   Sunil|
|  14|     Ram|
|  15|    Hari|
|  16|    John|
+----+--------+

